In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [50]:
training = pd.read_csv('TruckReidDataset/train.txt')
training['features'] = training['features'].apply(lambda x: np.array(eval(x)))
training.head()

,id,reid,timestamp,filepath,features,direction,drone,position_utm
0,2041,19.0,2025-09-09 13:43:00.333328247,train/02041.jpg,"[[-0.0032804568763822317, -0.02957571856677532...",Northbound,3,"[601044.64, 4859593.68]"
1,2471,65.0,2025-09-13 10:45:01.366668701,train/02471.jpg,"[[-0.005662569310516119, -0.03177795559167862,...",Southbound,1,"[604794.84, 4855944.94]"
2,2005,50.0,2025-09-09 17:56:19.533332825,train/02005.jpg,"[[-0.01376084890216589, -0.0229105893522501, -...",Southbound,2,"[602829.93, 4857858.49]"
3,1732,10.0,2025-09-09 10:30:34.466667175,train/01732.jpg,"[[-0.01391203235834837, -0.02876005880534649, ...",Northbound,2,"[602829.93, 4857858.49]"
4,2008,34.0,2025-09-09 16:29:21.866668701,train/02008.jpg,"[[-0.0010643844725564122, -0.00141269748564809...",Northbound,2,"[602829.93, 4857858.49]"
